In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [2]:
df = pd.read_csv("data/05-insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Regression with NN

In [3]:
features = ['age', 'bmi', 'children']
labels = ['charges']

X = df[features].values
y = df[labels].values

In [4]:
# Data standartization
FeaturScaler = StandardScaler()
LabelScaler = StandardScaler()
 
# Storing the fit object for later reference
FeaturScalerFit = FeaturScaler.fit(X)
LabelScalerFit = LabelScaler.fit(y)
 
# Generating the standardized values of X and y
X = FeaturScalerFit.transform(X)
y = LabelScalerFit.transform(y) 

In [5]:
# Split the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) 

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(936, 3)
(936, 1)
(402, 3)
(402, 1)


In [6]:
# Create model
model = Sequential()
model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(units=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 1, epochs = 5, verbose=0)
 
# Generating Predictions on testing data
Predictions = model.predict(X_test)
 
# Scaling the predicted Price data back to original price scale
Predictions = LabelScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original price scale
y_test_orig = LabelScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data = FeaturScalerFit.inverse_transform(X_test)
 
TestingData = pd.DataFrame(data=Test_Data, columns=features)
TestingData['charges'] = y_test_orig
TestingData['Predicted charges'] = Predictions
TestingData.head()

13/13 [==============================] - 0s 1ms/step


,age,bmi,children,charges,Predicted charges
0,45.0,25.175,2.0,9095.06825,12414.236328
1,36.0,30.020,0.0,5272.17580,11607.973633
2,64.0,26.885,0.0,29330.98315,16427.097656
3,46.0,25.745,3.0,9301.89355,13466.015625
4,19.0,31.920,0.0,33750.29180,8904.275391


In [7]:
# Computing the absolute percent error
errors = (TestingData['charges'] - TestingData['Predicted charges']) / TestingData['charges']
APE = 100 * abs(errors)
TestingData['APE'] = APE
 
print('The Accuracy of ANN model is:', round(100-np.mean(APE), 2), '%')

The Accuracy of ANN model is: -12.26 %


## Classification with NN

In [8]:
# Divide all records according to charges into three groups  
# and encode the groups
df["below_10000"] = df["charges"].apply(lambda x: 1 if x <= 10000 else 0)
df["over_30000"]  = df["charges"].apply(lambda x: 1 if x >= 30000 else 0)
df["between"]     = df["charges"].apply(lambda x: 1 if x > 10000 and x < 30000 else 0)
df.head()

,age,sex,bmi,children,smoker,region,charges,below_10000,over_30000,between
0,19,female,27.900,0,yes,southwest,16884.92400,0,0,1
1,18,male,33.770,1,no,southeast,1725.55230,1,0,0
2,28,male,33.000,3,no,southeast,4449.46200,1,0,0
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,0


In [9]:
X = df[["age", "bmi", "children"]]

# Values normalization
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

### Option #1

Three classes are coded with three output variables (1 0 0, 0 1 0, 0 0 1). In the output layer NN has three neurons

In [10]:
y = df[["below_10000", "over_30000", "between"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)

In [11]:
model = Sequential()
model.add(Dense(3, activation='relu', input_shape=(3,)))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.summary() 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 3)                 12        
                                                                 
 dense_4 (Dense)             (None, 6)                 24        
                                                                 
 dense_5 (Dense)             (None, 6)                 42        
                                                                 
 dense_6 (Dense)             (None, 3)                 21        
                                                                 
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=1, 
                    validation_data = (X_test, y_test))

Epoch 1/5
896/896 [==============================] - 3s 3ms/step - loss: 0.5952 - accuracy: 0.5915 - val_loss: 0.5516 - val_accuracy: 0.6629
Epoch 2/5
896/896 [==============================] - 2s 3ms/step - loss: 0.5318 - accuracy: 0.6562 - val_loss: 0.5005 - val_accuracy: 0.6900
Epoch 3/5
896/896 [==============================] - 2s 3ms/step - loss: 0.4989 - accuracy: 0.6931 - val_loss: 0.4693 - val_accuracy: 0.7172
Epoch 4/5
896/896 [==============================] - 2s 3ms/step - loss: 0.4775 - accuracy: 0.7031 - val_loss: 0.4513 - val_accuracy: 0.6991
Epoch 5/5
896/896 [==============================] - 2s 3ms/step - loss: 0.4626 - accuracy: 0.7143 - val_loss: 0.4309 - val_accuracy: 0.7489


In [13]:
model.evaluate(X_test,y_test)

14/14 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.7489


[0.430868923664093, 0.7488687634468079]

### Option #2

Three classes are coded with two output variables (1 0, 0 1, 0 0). In the output layer NN has two neurons

In [14]:
y = df[["below_10000", "over_30000"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)

In [15]:
model = Sequential()
model.add(Dense(3, activation='relu', input_shape=(3,)))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary() 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 3)                 12        
                                                                 
 dense_8 (Dense)             (None, 6)                 24        
                                                                 
 dense_9 (Dense)             (None, 6)                 42        
                                                                 
 dense_10 (Dense)            (None, 2)                 14        
                                                                 
Total params: 92
Trainable params: 92
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=1, 
                    validation_data = (X_test, y_test))

Epoch 1/5
896/896 [==============================] - 3s 3ms/step - loss: 0.5687 - accuracy: 0.8661 - val_loss: 0.5228 - val_accuracy: 0.8824
Epoch 2/5
896/896 [==============================] - 2s 3ms/step - loss: 0.5215 - accuracy: 0.8772 - val_loss: 0.5033 - val_accuracy: 0.8824
Epoch 3/5
896/896 [==============================] - 2s 3ms/step - loss: 0.5071 - accuracy: 0.8772 - val_loss: 0.4845 - val_accuracy: 0.8824
Epoch 4/5
896/896 [==============================] - 2s 3ms/step - loss: 0.4914 - accuracy: 0.8772 - val_loss: 0.4690 - val_accuracy: 0.8824
Epoch 5/5
896/896 [==============================] - 2s 3ms/step - loss: 0.4736 - accuracy: 0.8772 - val_loss: 0.4525 - val_accuracy: 0.8824


In [17]:
model.evaluate(X_test,y_test)

14/14 [==============================] - 0s 2ms/step - loss: 0.4525 - accuracy: 0.8824


[0.4525352120399475, 0.8823529481887817]